In [ ]:
from docx import Document

doc_path = "duden.docx"  
document = Document(doc_path)

# for paragraph in document.paragraphs:
#     print(paragraph.text)
for i in range(20):
    print(document.paragraphs[i].text)
    print("="*50)

In [11]:
# Word 파일 내의 표 읽기
for table in document.tables:
    for row in table.rows:
        for cell in row.cells:
            print(cell.text)  

## fitz

In [1]:
import fitz

pdf_file = "duden.pdf"
doc = fitz.open(pdf_file)

In [ ]:
extracted_text = []

# 32페이지부터 시작
for page in doc[::-1]:
    print('\n\n')
    print(f'==========================================PAGE {page}==========================================')
    for text in page.get_text("dict")["blocks"]:
        if "lines" in text:
            for line in text["lines"]:
                for span in line["spans"]: 
                    print(span)
                    extracted_text.append(span)

{'size': 7.5, 'flags': 4, 'font': 'Po.FrutigerCn-fett', 'color': 44527, 'ascender': 0.871999979019165, 'descender': -0.23999999463558197, 'text': ' kein Aas', 'origin': (207.72198486328125, 113.71902465820312), 'bbox': (207.72198486328125, 107.17902374267578, 237.33726501464844, 115.51902770996094)}

{'size': 7.800000190734863, 'flags': 6, 'font': 'Po.Kepler-kursiv', 'color': 2236191, 'ascender': 0.8539999723434448, 'descender': -0.2590000033378601, 'text': ' kein Mensch, niemand:', 'origin': (259.26019287109375, 113.71902465820312), 'bbox': (259.26019287109375, 107.05782318115234, 335.55755615234375, 115.73922729492188)}
{'size': 7.800000190734863, 'flags': 6, 'font': 'Po.Kepler-kursiv', 'color': 2236191, 'ascender': 0.8539999723434448, 'descender': -0.2590000033378601, 'text': ' clever sein, sich in allen', 'origin': (258.4385986328125, 176.71865844726562), 'bbox': (258.4385986328125, 170.05746459960938, 335.5882873535156, 178.73886108398438)}
{'size': 7.800000190734863, 'flags': 6, 'font': 'Po.Kepler-kursiv', 'color': 2236191, 'ascender': 0.8539999723434448, 'descender': -0.2590000033378601, 'text': 'Tricks und Schlichen auskennen:', 'origin': (200.31918334960938, 185.71829223632812), 'bbox': (200.31918334960938, 179.05709838867188, 306.0036926269531, 187.73849487304688)}
72.66949462890625
origin': (50.089149475097656

In [ ]:
import fitz  # PyMuPDF
import csv

# ==========================
# 1. PDF에서 조건에 맞는 스팬을 추출하여 rows 리스트 구성
# ==========================

# PDF 파일 경로 (필요에 따라 수정)
pdf_path = "duden.pdf"
doc = fitz.open(pdf_path)

# 조건 설정: 폰트 이름에 "fett" 포함, size가 7.5, color가 44527
target_font_keyword = "fett"
target_size = 7.5
target_color = 44527

# 조건에 맞는 텍스트를 저장할 리스트 (각 항목은 딕셔너리로 구성)
rows = []

for page in doc:
    page_dict = page.get_text("dict")
    if "blocks" not in page_dict:
        continue
    for block in page_dict["blocks"]:
        if "lines" not in block:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                font = span.get("font", "")
                size = span.get("size")
                color = span.get("color")
                # 조건 검사
                if target_font_keyword.lower() in font.lower() and size == target_size and color == target_color:
                    text = span.get("text", "").strip()
                    if text:  # 빈 문자열이 아니면
                        origin = span.get("origin", (0, 0))
                        rows.append({
                            "Redewendung": text,
                            "Bedeutung": "", 
                            "Beispiel": "",    
                            "origin_x": origin[0],
                            "origin_y": origin[1]
                        })
                        
                        print(span)

# ==========================
# 2. 'Redewendung' 값이 하이픈("-")으로 끝나면 다음 항목과 병합하고,
#    인접한 항목 중에서 origin_y 값이 같은(혹은 차이가 아주 미세한) 항목끼리도 병합 -> 같은 이디엄인데 따로 떨어져있는 애들 처리하는 것
# ==========================

merged_rows = []
i = 0
tolerance = 10.01  # y값 차이가 tolerance 이하이면 같은 라인으로 판단

while i < len(rows):
    # 현재 항목의 'Redewendung' 값 그대로 시작 (쉼표 제거는 필요하다면 rstrip(",") 사용)
    merged_text = rows[i]["Redewendung"].strip()
    
    # [1] 하이픈으로 끝나는 경우 다음 항목과 병합
    while merged_text.endswith("-") and (i + 1) < len(rows):
        # 하이픈(-) 제거 후, 다음 항목의 텍스트를 앞부분과 이어붙임
        merged_text = merged_text[:-1].rstrip() + rows[i + 1]["Redewendung"].lstrip()
        i += 1  # 다음 항목으로 이동
        
    # [2] 인접한 항목 중에서 origin_y 차이가 아주 미세한 경우 병합
    # (즉, 현재 항목과 바로 다음 항목이 같은 줄에 위치하는 경우)
    while (i + 1) < len(rows) and abs(rows[i + 1]["origin_y"] - rows[i]["origin_y"]) < tolerance:
        merged_text = merged_text + " " + rows[i + 1]["Redewendung"].lstrip()
        i += 1
    # 병합된 결과를 새로운 딕셔너리로 저장 (Bedeutung, Beispiel은 그대로 유지)
    merged_rows.append({
        "Redewendung": merged_text,
        "Bedeutung": rows[i]["Bedeutung"],
        "Beispiel": rows[i]["Beispiel"]
    })
    i += 1

# 결과 확인 (콘솔 출력)
print("=== 병합된 항목 ===")
for item in merged_rows:
    print("Redewendung:", item["Redewendung"])
    print("Bedeutung:", item["Bedeutung"])
    print("Beispiel:", item["Beispiel"])
    print("----------")

# ==========================
# 3. 전처리: merged_rows의 'Redewendung' 값에서 '"- "' (따옴표 없이 - 와 공백) 문자열 삭제
# ==========================
for row in merged_rows:
    # 만약 'Redewendung'에 "- " 가 포함되어 있으면 제거 (문자열 내 모든 발생 제거)
    row["Redewendung"] = row["Redewendung"].replace("- ", "")

# ==========================
# 4. CSV 파일로 저장 (칼럼: 'Redewendung', 'Bedeutung', 'Beispiel')
# ==========================

csv_file = "idioms.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["Redewendung", "Bedeutung", "Beispiel"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in merged_rows:
        writer.writerow(row)

print(f"\n총 {len(merged_rows)}개의 항목이 '{csv_file}' 파일에 저장되었습니다.")

## Bedeutung

In [ ]:
import fitz  # PyMuPDF
import csv
import math

# ========================================
# PDF 열기
# ========================================
pdf_path = "duden.pdf"
doc = fitz.open(pdf_path)

# ========================================
# 1. 조건에 맞는 Redewendung 및 Bedeutung 스팬 추출
# ========================================
# [Redewendung 조건]
rede_target_font_keyword = "fett"
rede_target_size = 7.5
rede_target_color = 44527

# [Bedeutung 조건]
bedeutung_target_font_keyword = "Po.Kepler-kursiv"
bedeutung_target_size = 7.8
bedeutung_target_color = 2236191
size_tolerance = 0.01  # 허용 오차

# 각각의 스팬을 저장할 리스트
rede_rows = []         # Redewendung 용
bedeutung_rows = []    # Bedeutung 용

# PDF의 각 페이지에 대해 텍스트 추출
for page in doc:
    page_number = page.number
    page_dict = page.get_text("dict")
    if "blocks" not in page_dict:
        continue
    for block in page_dict["blocks"]:
        if "lines" not in block:
            continue
        for line in block["lines"]:
            for span in line["spans"]:
                font = span.get("font", "")
                size = span.get("size")
                color = span.get("color")
                text = span.get("text", "").strip()
                origin = span.get("origin", (0, 0))
                
                # Redewendung 추출
                if rede_target_font_keyword.lower() in font.lower() and size == rede_target_size and color == rede_target_color:
                    if text:
                        rede_rows.append({
                            "Redewendung": text,
                            "Bedeutung": "", 
                            "Beispiel": "",
                            "origin_x": origin[0],
                            "origin_y": origin[1],
                            "page": page_number
                        })
                
                # Bedeutung 추출 (조건: 폰트, size, color, "↑" 미포함)
                if (bedeutung_target_font_keyword in font and 
                    math.isclose(size, bedeutung_target_size, abs_tol=size_tolerance) and 
                    color == bedeutung_target_color):
                    if "↑" in text:
                        continue
                    if text:
                        bedeutung_rows.append({
                            "Bedeutung": text,
                            "origin_x": origin[0],
                            "origin_y": origin[1],
                            "page": page_number
                        })

# ========================================
# 2. Redewendung 항목 병합 (하이픈 및 인접 항목 병합, 페이지별로 처리)
# ========================================
merged_rede = []
i = 0
tolerance = 10.01  # y좌표 병합 기준
while i < len(rede_rows):
    current = rede_rows[i]
    merged_text = current["Redewendung"].strip()
    current_page = current["page"]
    # 그룹의 첫번째 좌표 (병합 전 기준)
    group_origin_x = current["origin_x"]
    group_origin_y = current["origin_y"]
    
    # [1] 하이픈으로 끝나면 다음 항목과 병합 (같은 페이지 내에서)
    while merged_text.endswith("-") and (i + 1) < len(rede_rows) and rede_rows[i+1]["page"] == current_page:
        next_text = rede_rows[i+1]["Redewendung"].strip()
        merged_text = merged_text[:-1].rstrip() + next_text
        i += 1
        current = rede_rows[i]
    
    # [2] 인접 항목 병합 (같은 페이지, y좌표 차이가 미세한 경우)
    while (i + 1) < len(rede_rows) and rede_rows[i+1]["page"] == current_page and abs(rede_rows[i+1]["origin_y"] - current["origin_y"]) < tolerance:
        next_text = rede_rows[i+1]["Redewendung"].lstrip()
        merged_text = merged_text + " " + next_text
        i += 1
        current = rede_rows[i]
    
    merged_rede.append({
        "Redewendung": merged_text,
        "Bedeutung": current["Bedeutung"],  # 초기엔 빈 문자열
        "Beispiel": current["Beispiel"],
        "origin_x": group_origin_x,
        "origin_y": group_origin_y,
        "page": current_page
    })
    i += 1

# ========================================
# 3. Bedeutung 항목 병합 (Redewendung과 동일한 로직)
# ========================================
merged_bedeutung = []
i = 0
while i < len(bedeutung_rows):
    current = bedeutung_rows[i]
    merged_text = current["Bedeutung"].strip()
    current_page = current["page"]
    # 그룹의 첫번째 좌표 보존
    group_origin_x = current["origin_x"]
    group_origin_y = current["origin_y"]
    
    # 하이픈 처리
    while merged_text.endswith("-") and (i + 1) < len(bedeutung_rows) and bedeutung_rows[i+1]["page"] == current_page:
        next_text = bedeutung_rows[i+1]["Bedeutung"].strip()
        merged_text = merged_text[:-1].rstrip() + next_text
        i += 1
        current = bedeutung_rows[i]
    
    # 인접 항목 병합 (같은 페이지, y좌표 차이 미세)
    while (i + 1) < len(bedeutung_rows) and bedeutung_rows[i+1]["page"] == current_page and abs(bedeutung_rows[i+1]["origin_y"] - current["origin_y"]) < tolerance:
        next_text = bedeutung_rows[i+1]["Bedeutung"].lstrip()
        merged_text = merged_text + " " + next_text
        i += 1
        current = bedeutung_rows[i]
    
    merged_bedeutung.append({
        "Bedeutung": merged_text,
        "origin_x": group_origin_x,
        "origin_y": group_origin_y,
        "page": current_page
    })
    i += 1
# Bedeutung 항목 병합 후, 각 항목에 "used" 플래그 추가 (초기값 False)
for b in merged_bedeutung:
    b["used"] = False
# ========================================
# 4. 전처리: 병합된 텍스트에서 "- " (하이픈+공백) 삭제
# ========================================
for item in merged_rede:
    item["Redewendung"] = item["Redewendung"].replace("- ", "")
for item in merged_bedeutung:
    item["Bedeutung"] = item["Bedeutung"].replace("- ", "")

# ========================================
# 5. Redewendung과 Bedeutung 매칭 (여러 후보 중 가장 가까운 후보 선택)
#    조건: 같은 페이지 내에서
#          - y 좌표 차이가 10 이하인 후보들 중,
#            우선 Redewendung의 origin_x보다 큰(즉, 오른쪽에 있는) 후보들만 고려하여
#            그 중 dx(가로 차이)가 최소인 후보를 선택.
#            만약 오른쪽 후보가 없으면, 모든 후보 중에서 가장 가까운 후보를 선택.
# ========================================
unmatched_rede = []  # 매칭되지 않은 항목 저장

for rede in merged_rede:
    candidates = []
    # 같은 페이지에서 y 차이가 10 이하인 후보들 중, 아직 사용되지 않은 후보들을 추출
    for b in merged_bedeutung:
        if rede["page"] == b["page"] and not b.get("used", False):
        #if rede["page"] == b["page"]:
            if abs(rede["origin_y"] - b["origin_y"]) <= 10:
                candidates.append(b)
    best_candidate = None
    if candidates:
        # 우선, Redewendung의 origin_x보다 오른쪽(즉, b["origin_x"] > rede["origin_x"])에 있는 후보들을 선택
        right_candidates = [b for b in candidates if b["origin_x"] > rede["origin_x"]]
        if right_candidates:
            best_candidate = min(right_candidates, key=lambda b: abs(rede["origin_x"] - b["origin_x"]))
        else:
            # 오른쪽 후보가 없으면, 모든 후보 중에서 가장 가까운 후보 선택
            best_candidate = min(candidates, key=lambda b: abs(rede["origin_x"] - b["origin_x"]))
    
    if best_candidate is not None:
        rede["Bedeutung"] = best_candidate["Bedeutung"]
        best_candidate["used"] = True  # 이 후보는 이후에 재사용하지 않도록 표시
    else:
        unmatched_rede.append({
            "Redewendung": rede["Redewendung"],
            "page": rede["page"],
            "origin_x": rede["origin_x"],
            "origin_y": rede["origin_y"]
        })

# 매칭 실패한 항목들 좌표 출력
if unmatched_rede:
    print(f"{len(unmatched_rede)}개 Redewendung 항목에 대해 Bedeutung 매칭에 실패패했습니다.")
    print("==== 매칭 실패한 Redewendung 항목 ====")
    for item in unmatched_rede:
        print(f"Redewendung: {item['Redewendung']}")
        print(f"페이지: {item['page']}, origin_x: {item['origin_x']}, origin_y: {item['origin_y']}")
        print("----------------------------")
else:
    print("모든 Redewendung 항목에 대해 Bedeutung 매칭에 성공했습니다.")

# ========================================
# 6. 결과 CSV 파일로 저장 (칼럼: Redewendung, Bedeutung, Beispiel)
# ========================================
csv_file = "match_2.csv"
with open(csv_file, "w", newline="", encoding="utf-8") as csvfile:
    fieldnames = ["Redewendung", "Bedeutung", "Beispiel"]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for item in merged_rede:
        writer.writerow({
            "Redewendung": item["Redewendung"],
            "Bedeutung": item["Bedeutung"],
            "Beispiel": item["Beispiel"]
        })

print(f"\n총 {len(merged_rede)}개의 항목이 '{csv_file}' 파일에 저장되었습니다.")

In [ ]:
merged_bedeutung[30:400]

 {'Bedeutung': 'auf den Kopf:',
  'origin_x': 268.9902038574219,
  'origin_y': 59.719024658203125,
  'page': 908}
  {'Bedeutung': 'Alkohol trinken:',
  'origin_x': 162.37020874023438,
  'origin_y': 68.71902465820312,
  'page': 908},
  {'size': 7.5, 'flags': 4, 'font': 'Po.FrutigerCn-fett', 'color': 44527, 'ascender': 0.871999979019165, 'descender': -0.23999999463558197, 'text': 'auf die Zwölf', 'origin': (203.1595001220703, 59.719024658203125), 'bbox': (203.1595001220703, 53.17902374267578, 246.97300720214844, 61.51902389526367)}


# row 채우기

In [ ]:
import fitz
import re

def is_close(a, b, tol=0.3):
    """
    a와 b의 차이가 tol 이하이면 True를 반환합니다.
    (예를 들어 tol=0.3이면 a와 b가 0.3 이내 차이면 같은 것으로 간주)
    """
    return abs(a - b) < tol

pdf_file = "duden.pdf"
doc = fitz.open(pdf_file)
extracted_text = []

# PDF의 모든 페이지(여기서는 역순으로 처리)에서 span 데이터 추출
for page in doc[31:]:
    for block in page.get_text("dict")["blocks"]:
        if "lines" in block:
            for line in block["lines"]:
                for span in line["spans"]:
                    extracted_text.append(span)
                    print(span)
#print(extracted_text[20:24])

In [ ]:
import re

def is_close(a, b, tol=0.3):
    """ a와 b의 차이가 tol 이하이면 True (글자 크기 비교용) """
    return abs(a - b) < tol

# 원치 않는 텍스트 목록
EXCLUDE_TEXTS = {"↑", "(ugs.):", " (ugs.):"}

# 그룹화 단계
groups = []
current_group = None
current_redewendung = ""  # 관용구 연결을 위한 변수

for span in extracted_text:
    font = span.get("font", "")
    size = span.get("size", 0)
    color = span.get("color", None)
    text = span.get("text", "").strip()
    
    if not text or text in EXCLUDE_TEXTS:
        continue  # 빈 텍스트 또는 제외할 텍스트는 건너뜀

    # [Redewendung] 감지 (관용구)
    if ("FrutigerCn-fett" in font and "FrutigerCn-extrafett" not in font 
            and is_close(size, 7.5) and color == 44527):
        if current_redewendung:
            # 이전 관용구가 있다면 이어 붙이기
            current_redewendung += " " + text
        else:
            # 새로운 관용구 시작
            if current_group:
                groups.append(current_group)
            current_redewendung = text
            current_group = {"redewendung": current_redewendung, "bedeutung": [], "beispiele": []}

    # [Bedeutung] 감지 (의미)
    elif "Kepler-kursiv" in font and is_close(size, 7.8):
        if current_group:
            current_group["bedeutung"].append(text)

    # [Beispiel] 감지 (예제)
    elif "Kepler-normal" in font and is_close(size, 7.8):
        if current_group:
            current_group["beispiele"].append(text)

# 마지막 그룹 저장
if current_group:
    groups.append(current_group)

# ------------------------------------------------------------------
# Bedeutung 후처리 (문장 연결 및 번호 분리)
def process_bedeutung(bedeutung_list):
    result = []
    current = ""
    for text in bedeutung_list:
        # "1.", "2." 등으로 시작하면 새로운 의미로 분리
        if re.match(r"^\s*\d+\.", text):
            if current:
                result.append(current.strip())
            current = text.strip()
        else:
            if current:
                current += " " + text.strip()
            else:
                current = text.strip()
    
    if current:
        result.append(current.strip())
    return result

# ------------------------------------------------------------------
# 최종 데이터 생성
rows = []
for group in groups:
    processed_bedeutungen = process_bedeutung(group["bedeutung"])
    beispiele_joined = " ".join(group["beispiele"]).strip()

    if len(processed_bedeutungen) > 1:
        for b in processed_bedeutungen:
            rows.append({
                "Redewendung": group["redewendung"],
                "Bedeutung": b,
                "Beispiel": beispiele_joined
            })
    else:
        bedeutung_text = processed_bedeutungen[0] if processed_bedeutungen else ""
        rows.append({
            "Redewendung": group["redewendung"],
            "Bedeutung": bedeutung_text,
            "Beispiel": beispiele_joined
        })

# ------------------------------------------------------------------
# 최종 데이터 출력
def print_results(rows):
    print("\n===== 정제된 데이터 =====")
    for i, row in enumerate(rows, start=1):
        print(f"\n[{i}]")
        print(f"🔹 Redewendung: {row['Redewendung']}")
        print(f"🔸 Bedeutung  : {row['Bedeutung']}")
        print(f"🔹 Beispiel   : {row['Beispiel']}")
        print("-" * 50)

# 결과 출력
print_results(rows)

In [ ]:
# ------------------------------------------------------------------
# 최종 데이터 출력 함수
def print_results(rows):
    print("\n===== 정제된 데이터 =====")
    for i, row in enumerate(rows, start=1):
        print(f"\n[{i}]")
        print(f"🔹 Redewendung: {row['Redewendung']}")
        print(f"🔸 Bedeutung  : {row['Bedeutung']}")
        print(f"🔹 Beispiel   : {row['Beispiel']}")
        print("-" * 50)

# 결과 출력
print_results(rows)